<a href="https://colab.research.google.com/github/nematatu/Colab-Python/blob/master/%E6%84%9F%E6%83%85%E3%81%AB%E3%82%88%E3%82%8B%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%81%AE%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
#ファイルの作成、ディレクトリの移動とか、OS依存の機能をいじるのに使う
import os
#re(Regular Expressions)正規表現を扱うのに使う
#文字列パターンを検索、抽出、置換したり
import re
#ファイル、ディレクトリの操作をするのに使う
#osと何が違うんだ↓
#shutilはosを基に、高度なファイル、ディレクトリ操作ができるようになったものらしい
import shutil
#文字列の操作に使う
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
print(tf.__version__)

2.14.0


In [3]:
#IMDB(Internet Movie DataBase)という50000件の映画レビューから抽出した、データセットを使う
#トレーニング用とテスト用に25000件ずつある
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
#↑urlからファイルをダウンロードする
#tf.keras.utils.get_file()
#"aclImdb": 保存先のディレクトリ名
#url: urlの指定
#untar: 保存するファイルが.tarなら、解凍するかどうか
#cache_dir: ファイルのダウンロード先のディレクトリを指定
#cache_subdir: 保存先のサブディレクトリを指定。今回はなし
dataset=tf.keras.utils.get_file("aclImdb",url,untar=True,cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset),'aclImdb')

84125825/84125825 [==============================] - 11s 0us/step


In [4]:
print(os.listdir(os.path.dirname(dataset)))
os.listdir(dataset_dir)

['.config', 'aclImdb.tar.gz', 'aclImdb', 'sample_data']


['README', 'train', 'test', 'imdb.vocab', 'imdbEr.txt']

In [5]:
train_dir=os.path.join(dataset_dir,'train')
print(os.listdir(train_dir))

['unsupBow.feat', 'unsup', 'labeledBow.feat', 'pos', 'urls_pos.txt', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


In [6]:
sample_file=os.path.join(train_dir,'pos/1189_9.txt')
#with: 安全にファイル操作ができるらしい
with open(sample_file) as f:
  print(f.read())

The Last Hunt is one of the few westerns ever made to deal with Buffalo hunting, both as a sport and business and as a method of winning the plains Indian wars. Before the white man set foot on the other side of the Mississippi, the plains used to have herds of American Bison as large as some of our largest cities. By the time of the period The Last Hunt is set in, the buffalo had been all but wiped out. The 20th century, due to the efforts of conservationists, saw a revival in population of the species, but not hardly like it once was.<br /><br />Robert Taylor and Stewart Granger are co-starring in a second film together and this one is far superior to All the Brothers Were Valiant. Here Stewart Granger is the good guy, a world weary buffalo hunter, who has to go back to a job he hates because of financial considerations.<br /><br />The partner he's chosen to throw in with is Robert Taylor. Forgetting Taylor for the moment, I doubt if there's ever been a meaner, nastier soul than Char

In [8]:
remove_dir=os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

In [10]:
batch_size=32
seed=42

#元のディレクトリから、テキスト形式のデータセットにする
raw_train_ds=tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    #内20%を検証用に使う
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [40]:
print(raw_train_ds.class_names)

['neg', 'pos']


In [39]:
for text_batch,label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review",text_batch.numpy()[i])
    print("Label",label_batch.numpy()[i])

Review b'I try to be very objective when I view a low budget movie. I also apply a lower weight to independent and low budget productions versus the big budget productions. I expect near flawlessness from big budget productions and their studios. Therefore I apply tougher criteria to the major studio releases. But this movie was just a dud. Period. The premise was terrible. The main character, Mary Gordano (Alicia Silverstone), was unbelievable as a high school senior with an unquenchable desire to solve crimes. There was not enough depth in her character or her acting that pulled you into her world. Also, to make this movie more mysterious, the lighting in certain scenes did not set the mood, especially in the warehouse.<br /><br />Once again another disappointing movie that I could only give three points to.'
Label 0
Review b'Have you ever, or do you have, a pet who\'s been with you through thick and thin, who you\'d be lost without, and who you love no matter what? Betcha never thou

In [41]:
print("label 0 correspond to ",raw_train_ds.class_names[0])
print("label 1 correspond to ",raw_train_ds.class_names[1])

label 0 correspond to  neg
label 1 correspond to  pos
